In [ ]:
#!pip install dippykit

In [11]:
import numpy as np
import cv2
import zmq
import time
from statistics import stdev
from utils import *
from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode
import pickle

In [27]:
x = np.array([1, 2, 3, 4, 5])
print(x.shape)
encoded_x, stream_length, code_dict, prob_dict = huffman_encode(x)
print(type(code_dict))
decoded_x = huffman_decode(encoded_x, code_dict, stream_length)
print(decoded_x)

(5,)
<class 'dict'>
[1 2 3 4 5]


In [52]:
def send_full_vector(n=5):
    video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()

    count = 1
    processing_times = []
    sending_times = []
    total_start = time.time()
    while success:
        print(count)
        if count%n == 0:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        V = R.sample(img, fixation).astype('uint8')

        start = time.time()
        V = V.tostring()
        send_start = time.time()
        tracker = socket.send(V, copy=False, track=True)
        while tracker.done == False:
            pass
            
        end = time.time()
        processing_times.append(send_start-start)
        sending_times.append(end-send_start)
        
        success, img = vidcap.read()
        count += 1
    total_end = time.time()
    
    print('Successfully converted video to %d frames'%count)
    
    print('\nProcessing time: %f'%sum(processing_times))
    avg = sum(processing_times)/len(processing_times)
    std = stdev(processing_times)
    #print('Average time taken: {avg:.5f} ± {std:.5f}'.format(avg=avg, std=std))
    
    print('Time taken to send: %f'%sum(sending_times))
    avg = sum(sending_times)/len(sending_times)
    std = stdev(sending_times)
    #print('Average time taken to send: {avg:.5f} ± {std:.5f}'.format(avg=avg, std=std))

    print('Overall total time: %f'%(total_end-total_start))

def send_encoded_vector(n):
    video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()

    count = 1
    processing_times = []
    sending_times = []
    total_start = time.time()
    while success:
        print(count)
        if count%n == 0:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        V = R.sample(img, fixation).astype('uint8')
        start = time.time()
        
        encoded_V, bits, code_dict, _ = huffman_encode(V.flatten())
        #data = [encoded_V, code_dict, bits]
        data = encoded_V.tostring()#pickle.dumps(data, protocol=-1)
        
        sending_start = time.time()
        tracker = socket.send(data, copy=False, track=True)
        while tracker.done == False:
            pass

        end = time.time()
        processing_times.append(sending_start-start)
        sending_times.append(end-sending_start)
        success, img = vidcap.read()
        count += 1
    total_end = time.time()
    print('Successfully converted video to %d frames'%count)
    
    print('\nProcessing time: %f'%sum(processing_times))
    #avg = sum(processing_times)/len(processing_times)
    #std = stdev(processing_times)
    #print('Average time taken: {avg:.5f} ± {std:.5f}'.format(avg=avg, std=std))
    print('Time taken to send: %f'%sum(sending_times))
    print('Overall total time: %f'%(total_end-total_start))

In [29]:
R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1920/2, 1080/2)
shape = (1920, 1080)
R.prepare(shape, fixation)

In [40]:
n = 25

In [54]:
context = zmq.Context()

#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")

send_full_vector(n)

socket.close()

Connecting to Server…
1


/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:109: RuntimeWarning: divide by zero encountered in true_divide
  if rgb: f = 1.0/np.sum(m*kernel, axis = (0,1)) #TODO fix invalid value warnings
/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:113: RuntimeWarning: invalid value encountered in multiply
  if rgb: V[i] = np.sum(extract*kernel, axis=(0,1)) * f


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Successfully converted video to 25 frames

Processing time: 0.000562
Time taken to send: 0.157380
Overall total time: 82.054872


In [44]:
context = zmq.Context()
#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")

send_encoded_vector(n)

socket.close()

Connecting to Server…
1


/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:109: RuntimeWarning: divide by zero encountered in true_divide
  if rgb: f = 1.0/np.sum(m*kernel, axis = (0,1)) #TODO fix invalid value warnings
/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:113: RuntimeWarning: invalid value encountered in multiply
  if rgb: V[i] = np.sum(extract*kernel, axis=(0,1)) * f


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Successfully converted video to 25 frames

Processing time: 2.234727
Time taken to send: 0.160053
Overall total time: 79.575132


In [53]:
context = zmq.Context()
#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")

send_encoded_vector(n)

socket.close()

Connecting to Server…
1


/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:109: RuntimeWarning: divide by zero encountered in true_divide
  if rgb: f = 1.0/np.sum(m*kernel, axis = (0,1)) #TODO fix invalid value warnings
/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/utils.py:113: RuntimeWarning: invalid value encountered in multiply
  if rgb: V[i] = np.sum(extract*kernel, axis=(0,1)) * f


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Successfully converted video to 25 frames

Processing time: 2.324750
Time taken to send: 0.151740
Overall total time: 85.114513


In [1]:
import zmq
import time

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")

#  Do 10 requests, waiting each time for a response
for request in range(10):
    print(f"Sending request {request} …")
    socket.send(b"Hello")

    #  Get the reply.
    message = socket.recv()
    print(f"Received reply {request} [ {message} ]")


Connecting to hello world server…
Sending request 0 …
Received reply 0 [ b'World' ]
Sending request 1 …
Received reply 1 [ b'World' ]
Sending request 2 …
Received reply 2 [ b'World' ]
Sending request 3 …
Received reply 3 [ b'World' ]
Sending request 4 …
Received reply 4 [ b'World' ]
Sending request 5 …
Received reply 5 [ b'World' ]
Sending request 6 …
Received reply 6 [ b'World' ]
Sending request 7 …
Received reply 7 [ b'World' ]
Sending request 8 …
Received reply 8 [ b'World' ]
Sending request 9 …
Received reply 9 [ b'World' ]
